# Démonstration : Message Authentication Codes (MAC)

**Objectifs** :
- Comprendre le rôle des MAC (authentification sans confidentialité)
- Implémenter HMAC-SHA256 (standard industriel)
- Comparer avec CBC-MAC
- Démontrer les attaques sur MAC mal conçus

In [ ]:
import hashlib
import hmac
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import time

## 1. Pourquoi les MAC ?

**Problème** : Alice envoie un message à Bob. Comment Bob peut-il vérifier :
1. Le message vient bien d'Alice (authentification)
2. Le message n'a pas été modifié (intégrité)

**Solution** : Message Authentication Code (MAC)

**Définition** : $\text{MAC} : \mathcal{K} \times \mathcal{M} \to \mathcal{T}$
- Alice calcule $t = \text{MAC}_k(m)$ et envoie $(m, t)$
- Bob vérifie $\text{Verify}_k(m, t) \stackrel{?}{=} \text{true}$

## 2. HMAC (Hash-based MAC) - Standard Industriel

**Construction** : $\text{HMAC}_k(m) = H((k \oplus opad) \| H((k \oplus ipad) \| m))$

où :
- $H$ : fonction de hachage (SHA-256, SHA-3, etc.)
- $ipad = 0x36$ répété (inner padding)
- $opad = 0x5c$ répété (outer padding)

**Avantages** :
- Sécurisé si $H$ est résistant aux collisions
- Rapide (pas de chiffrement)
- Standard : HMAC-SHA256, HMAC-SHA3

In [ ]:
def hmac_demo():
    """
    Démonstration complète de HMAC-SHA256.
    """
    # Clé secrète partagée entre Alice et Bob
    key = secrets.token_bytes(32)  # 256 bits
    
    print("=" * 70)
    print("HMAC-SHA256 : Authentification de Messages")
    print("=" * 70)
    print(f"\nClé secrète (hex) : {key.hex()}")
    
    # Message à authentifier
    message = b"Transfer $1000 from Alice to Bob"
    print(f"\nMessage : {message}")
    
    # Génération du MAC (côté Alice)
    mac_tag = hmac.new(key, message, hashlib.sha256).digest()
    
    print(f"\n👩 Alice calcule le MAC :")
    print(f"   TAG (hex) : {mac_tag.hex()}")
    print(f"   Taille    : {len(mac_tag)} bytes = {len(mac_tag)*8} bits")
    
    # Alice envoie (message, mac_tag)
    print(f"\n📤 Alice envoie : (message, TAG)")
    
    # Vérification du MAC (côté Bob)
    print(f"\n👨 Bob vérifie le MAC :")
    
    # Bob recalcule le MAC
    expected_mac = hmac.new(key, message, hashlib.sha256).digest()
    
    # Comparaison sécurisée (constant-time)
    is_valid = hmac.compare_digest(mac_tag, expected_mac)
    
    print(f"   MAC recalculé : {expected_mac.hex()}")
    print(f"   Vérification  : {'✅ VALIDE' if is_valid else '❌ INVALIDE'}")
    
    return key, message, mac_tag

key, message, mac_tag = hmac_demo()

## 3. Attaque : Modification du Message

Un attaquant (Eve) essaie de modifier le message sans connaître la clé.

In [ ]:
def mac_tampering_attempt():
    """
    Eve tente de modifier le message.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Modification du Message")
    print("=" * 70)
    
    # Eve intercepte (message, mac_tag)
    original_message = message
    original_mac = mac_tag
    
    print(f"\n🕵️ Eve intercepte :")
    print(f"   Message : {original_message}")
    print(f"   MAC     : {original_mac.hex()[:32]}...")
    
    # Eve modifie le message
    modified_message = b"Transfer $9999 from Alice to Eve"  # 🏴‍☠️
    
    print(f"\n💀 Eve modifie le message :")
    print(f"   Nouveau : {modified_message}")
    print(f"   MAC     : {original_mac.hex()[:32]}... (inchangé)")
    
    # Bob vérifie le MAC modifié
    print(f"\n👨 Bob vérifie le MAC :")
    expected_mac = hmac.new(key, modified_message, hashlib.sha256).digest()
    is_valid = hmac.compare_digest(original_mac, expected_mac)
    
    print(f"   MAC attendu  : {expected_mac.hex()[:32]}...")
    print(f"   MAC reçu     : {original_mac.hex()[:32]}...")
    print(f"   Vérification : {'✅ VALIDE' if is_valid else '❌ INVALIDE (REJETÉ)'}")
    
    if not is_valid:
        print(f"\n✅ Attaque DÉTECTÉE et BLOQUÉE !")
        print(f"   Bob rejette le message modifié.")

mac_tampering_attempt()

## 4. Attaque : MAC Forgery (sans clé)

**Question** : Eve peut-elle créer un MAC valide sans connaître la clé ?

**Réponse** : Non, si HMAC est utilisé correctement avec une fonction de hachage sécurisée.

In [ ]:
def mac_forgery_attempt():
    """
    Eve tente de forger un MAC sans connaître la clé.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Forgery (création de MAC sans clé)")
    print("=" * 70)
    
    # Eve veut créer un message avec un MAC valide
    eve_message = b"Transfer $5000 from Bob to Eve"
    
    print(f"\n💀 Eve veut authentifier :")
    print(f"   Message : {eve_message}")
    
    # Tentative 1 : MAC aléatoire
    print(f"\n🎲 Tentative 1 : MAC aléatoire")
    random_mac = secrets.token_bytes(32)
    expected_mac = hmac.new(key, eve_message, hashlib.sha256).digest()
    success = hmac.compare_digest(random_mac, expected_mac)
    print(f"   MAC forgé : {random_mac.hex()[:32]}...")
    print(f"   Résultat  : {'✅ SUCCÈS' if success else '❌ ÉCHEC (attendu)'}")
    
    # Tentative 2 : Brute force (infaisable)
    print(f"\n💪 Tentative 2 : Brute force")
    print(f"   Espace de clés : 2^256 (256 bits)")
    print(f"   Temps estimé   : ~10^77 secondes (âge de l'univers : ~10^18 s)")
    print(f"   Conclusion     : ❌ INFAISABLE")
    
    # Tentative 3 : Hash seul (sans clé) - VULNÉRABLE !
    print(f"\n⚠️  Tentative 3 : Si MAC = H(m) (sans clé) ?")
    print(f"   → Eve peut calculer H(m) pour n'importe quel message !")
    print(f"   → C'est pourquoi on utilise HMAC avec une CLÉ SECRÈTE")
    
    weak_mac = hashlib.sha256(eve_message).digest()
    print(f"   H(m) = {weak_mac.hex()[:32]}...")
    print(f"   ❌ VULNÉRABLE : Ne JAMAIS utiliser H(m) seul comme MAC !")

mac_forgery_attempt()

## 5. CBC-MAC - Alternative pour Block Ciphers

**Construction** : Chaîner les blocs avec CBC, garder le dernier bloc comme MAC.

$$\text{CBC-MAC}_k(m) = E_k(E_k(m_1 \oplus 0) \oplus m_2 \oplus \ldots)$$

**Avantages** : Basé sur AES (si accélération matérielle)

**Inconvénients** :
- ⚠️ Vulnérable si messages de longueur variable non gérés
- ⚠️ Nécessite padding

**Recommandation** : Préférer HMAC ou AEAD (AES-GCM)

In [ ]:
def cbc_mac_demo():
    """
    Implémentation simplifiée de CBC-MAC.
    """
    print("\n" + "=" * 70)
    print("CBC-MAC (AES-128)")
    print("=" * 70)
    
    key = secrets.token_bytes(16)  # AES-128
    message = b"Authenticated message with CBC-MAC"
    
    # Padding PKCS7
    padder = padding.PKCS7(128).padder()
    padded = padder.update(message) + padder.finalize()
    
    # CBC avec IV = 0
    iv = b'\x00' * 16
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded) + encryptor.finalize()
    
    # MAC = dernier bloc
    mac_tag = ciphertext[-16:]
    
    print(f"\nMessage  : {message}")
    print(f"Padded   : {len(padded)} bytes")
    print(f"MAC (hex): {mac_tag.hex()}")
    print(f"Taille   : {len(mac_tag)} bytes = {len(mac_tag)*8} bits")
    
    # Vérification
    cipher_verify = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor_verify = cipher_verify.encryptor()
    ciphertext_verify = encryptor_verify.update(padded) + encryptor_verify.finalize()
    mac_verify = ciphertext_verify[-16:]
    
    is_valid = hmac.compare_digest(mac_tag, mac_verify)
    print(f"\nVérification : {'✅ VALIDE' if is_valid else '❌ INVALIDE'}")
    
    return key, message, mac_tag

cbc_mac_demo()

## 6. Attaque sur CBC-MAC : Length Extension

**Vulnérabilité** : Si CBC-MAC utilisé naïvement sans encodage de longueur.

**Attaque** : À partir de $(m, t)$ valide, forger $(m \| m', t')$ valide.

In [ ]:
def cbc_mac_length_extension_attack():
    """
    Démonstration d'attaque par extension sur CBC-MAC mal implémenté.
    
    ⚠️ Cette attaque fonctionne sur CBC-MAC SANS encodage de longueur.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Length Extension sur CBC-MAC")
    print("=" * 70)
    
    key = secrets.token_bytes(16)
    
    # Message original de 1 bloc (16 bytes)
    m1 = b"Pay Eve $100.00!"  # Exactement 16 bytes
    
    # CBC-MAC de m1
    iv = b'\x00' * 16
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    t1 = encryptor.update(m1) + encryptor.finalize()
    
    print(f"\n✅ Message authentifié original :")
    print(f"   m1 : {m1}")
    print(f"   t1 : {t1.hex()}")
    
    # Eve veut ajouter un bloc m2
    m2 = b"...and $900 more"  # 16 bytes
    
    # Attaque : m2' = m2 ⊕ t1
    m2_xor_t1 = bytes(a ^ b for a, b in zip(m2, t1))
    
    # Message forgé : m1 || m2'
    forged_message = m1 + m2_xor_t1
    
    print(f"\n💀 Eve forge un message étendu :")
    print(f"   m_forged : {m1} || {m2_xor_t1.hex()}")
    
    # Calculer MAC du message forgé
    cipher_forge = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor_forge = cipher_forge.encryptor()
    t_forged = encryptor_forge.update(forged_message) + encryptor_forge.finalize()
    t_forged_final = t_forged[-16:]  # Dernier bloc
    
    # Calculer MAC de m2 seul avec IV = t1
    cipher_m2 = Cipher(algorithms.AES(key), modes.CBC(t1), backend=default_backend())
    encryptor_m2 = cipher_m2.encryptor()
    t2 = encryptor_m2.update(m2) + encryptor_m2.finalize()
    
    print(f"\n🔍 Vérification :")
    print(f"   MAC(m1||m2') calculé : {t_forged_final.hex()}")
    print(f"   CBC-MAC(m2, IV=t1)   : {t2.hex()}")
    print(f"   Identique ? {t_forged_final == t2} {'✅' if t_forged_final == t2 else '❌'}")
    
    print(f"\n⚠️  VULNÉRABILITÉ : CBC-MAC sans encodage de longueur")
    print(f"✅ SOLUTION : Utiliser CMAC (RFC 4493) ou HMAC")

cbc_mac_length_extension_attack()

## 7. Comparaison HMAC vs CBC-MAC

In [ ]:
def benchmark_mac():
    """
    Compare les performances de HMAC vs CBC-MAC.
    """
    print("\n" + "=" * 70)
    print("BENCHMARK : HMAC-SHA256 vs CBC-MAC-AES")
    print("=" * 70)
    
    key_hmac = secrets.token_bytes(32)
    key_cbc = secrets.token_bytes(16)
    
    sizes = [1024, 10240, 102400]  # 1KB, 10KB, 100KB
    iterations = 1000
    
    print(f"\nItérations : {iterations} par taille\n")
    
    for size in sizes:
        data = secrets.token_bytes(size)
        
        # HMAC-SHA256
        start = time.perf_counter()
        for _ in range(iterations):
            hmac.new(key_hmac, data, hashlib.sha256).digest()
        time_hmac = time.perf_counter() - start
        
        # CBC-MAC (simplifié)
        padder = padding.PKCS7(128).padder()
        padded = padder.update(data) + padder.finalize()
        
        start = time.perf_counter()
        for _ in range(iterations):
            iv = b'\x00' * 16
            cipher = Cipher(algorithms.AES(key_cbc), modes.CBC(iv), backend=default_backend())
            encryptor = cipher.encryptor()
            encryptor.update(padded) + encryptor.finalize()
        time_cbc = time.perf_counter() - start
        
        throughput_hmac = (size * iterations) / time_hmac / 1_000_000
        throughput_cbc = (size * iterations) / time_cbc / 1_000_000
        
        print(f"Taille : {size:>6} bytes ({size//1024:>3} KB)")
        print(f"  HMAC-SHA256 : {time_hmac:>6.3f}s ({throughput_hmac:>6.1f} MB/s)")
        print(f"  CBC-MAC-AES : {time_cbc:>6.3f}s ({throughput_cbc:>6.1f} MB/s)")
        print()
    
    print("Note : CBC-MAC peut être plus rapide avec accélération AES-NI")

benchmark_mac()

## 8. Timing Attack sur Vérification de MAC

**Vulnérabilité** : Comparer les MAC byte par byte révèle des informations !

**Attaque** : Mesurer le temps de vérification pour deviner le MAC.

In [ ]:
def insecure_verify(mac1: bytes, mac2: bytes) -> bool:
    """
    Vérification VULNÉRABLE (byte par byte).
    
    ⚠️ NE JAMAIS utiliser en production !
    """
    if len(mac1) != len(mac2):
        return False
    
    for a, b in zip(mac1, mac2):
        if a != b:
            return False  # Sort dès la première différence !
    return True

def secure_verify(mac1: bytes, mac2: bytes) -> bool:
    """
    Vérification SÉCURISÉE (constant-time).
    
    ✅ Toujours utiliser hmac.compare_digest() en pratique.
    """
    if len(mac1) != len(mac2):
        return False
    
    result = 0
    for a, b in zip(mac1, mac2):
        result |= a ^ b  # XOR tous les bytes
    return result == 0

# Démonstration
print("\n" + "=" * 70)
print("TIMING ATTACK : Vérification de MAC")
print("=" * 70)

correct_mac = secrets.token_bytes(32)

# Test 1 : Premier byte correct
guess1 = bytes([correct_mac[0]]) + secrets.token_bytes(31)

# Test 2 : Premier byte incorrect
guess2 = bytes([correct_mac[0] ^ 0xFF]) + secrets.token_bytes(31)

print(f"\n⚠️  Vérification INSÉCURISÉE (byte-by-byte) :")

# Mesurer temps (simplifié, en pratique nécessite millions d'essais)
start = time.perf_counter()
for _ in range(100000):
    insecure_verify(correct_mac, guess1)
time1 = time.perf_counter() - start

start = time.perf_counter()
for _ in range(100000):
    insecure_verify(correct_mac, guess2)
time2 = time.perf_counter() - start

print(f"  1er byte correct   : {time1:.6f}s")
print(f"  1er byte incorrect : {time2:.6f}s")
print(f"  Différence         : {abs(time1-time2):.6f}s")

if abs(time1 - time2) > 0.0001:
    print(f"  ⚠️  Temps différents → VULNÉRABLE au timing attack !")

print(f"\n✅ Vérification SÉCURISÉE (constant-time) :")

start = time.perf_counter()
for _ in range(100000):
    secure_verify(correct_mac, guess1)
time3 = time.perf_counter() - start

start = time.perf_counter()
for _ in range(100000):
    secure_verify(correct_mac, guess2)
time4 = time.perf_counter() - start

print(f"  1er byte correct   : {time3:.6f}s")
print(f"  1er byte incorrect : {time4:.6f}s")
print(f"  Différence         : {abs(time3-time4):.6f}s")
print(f"  ✅ Temps constants → Résistant au timing attack")

print(f"\n💡 Toujours utiliser hmac.compare_digest() en Python !")

## Conclusion

**Points clés** :
- MAC garantit authentification + intégrité (pas confidentialité)
- ✅ HMAC-SHA256 : Standard industriel recommandé
- ⚠️ CBC-MAC : Vulnérable si mal implémenté (length extension)
- ❌ Ne JAMAIS utiliser H(m) seul comme MAC
- ✅ Toujours utiliser comparaison constant-time (timing attacks)

**En pratique** :
- Pour authentification seule : **HMAC-SHA256**
- Pour chiffrement + authentification : **AEAD** (AES-GCM, ChaCha20-Poly1305)
- TLS 1.3 utilise : AEAD exclusivement
- SSH utilise : HMAC ou Encrypt-then-MAC